In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, classification_report
import pickle
import os
import glob
import seaborn as sns
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn import preprocessing
from sklearn.utils import class_weight
from sklearn.model_selection import cross_val_score,StratifiedKFold
import xgboost as xgb

In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [6]:
i = 1
frames = []
for name in os.listdir('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/client_data'):
    extents = os.path.splitext('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/client_data'+'/{}'.format(name))[1].lower()
    if extents == '.csv':
        df= pd.read_csv('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/client_data/{}'.format(name))
        frames.append(df)
        print('csv',df.shape)
    elif extents == '.xlsx':
        df= pd.read_excel('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/client_data/{}'.format(name))
        df.columns = df.columns.astype(str)
        frames.append(df)
        print('xlx',df.shape)

result =  pd.concat(frames, axis=0, ignore_index= True)

csv (78, 1060)
csv (191, 1060)
csv (132, 1059)
csv (224, 1060)
csv (42, 1060)
csv (142, 1060)
csv (186, 1060)
csv (80, 1060)
csv (348, 1060)
csv (102, 1060)
csv (198, 1060)
csv (222, 1060)
csv (110, 1060)
xlx (680, 1059)
csv (250, 1060)
xlx (224, 1059)
csv (92, 1060)
csv (80, 1060)
csv (322, 1060)


In [7]:
result.head()

,,L*,a*,b*,X,Y,Z,Dominant Wavelenght,Whiteness,Purity,...,979.335,980.054,980.772,981.491,982.209,982.928,983.646,984.364,985.082,Unnamed: 1059
0,W_Calibration,100.000,0.000000,0.000000,95.0422,100.000,108.8760,565.016,99.9909,0.000017,...,100.0000,100.0000,100.000,100.0000,100.000,100.00000,100.0000,100.0000,100.0000,NaN
1,F_Calibration,100.000,0.000000,0.000000,108.1040,100.000,39.2965,491.070,99.9909,0.000170,...,100.0000,100.0000,100.000,100.0000,100.000,100.00000,100.0000,100.0000,100.0000,NaN
2,W_VK12 | 2_J_None_0.902_SI1,101.352,-1.569150,4.040870,97.4907,103.537,106.1060,570.000,86.0505,0.035779,...,106.0810,106.2960,106.206,106.8210,106.483,91.47060,120.7140,27.2222,148.3330,NaN
3,F_VK12 | 2_J_None_0.902_SI1,100.595,3.739330,-10.586200,112.2450,101.547,46.5464,575.670,140.8850,0.041247,...,62.4801,62.8836,62.791,62.8042,62.124,51.55710,38.7013,56.8270,-143.2990,NaN
4,W_3472A2 | 126_D_None_0.702_VS1,100.909,-0.104228,-0.133574,97.2349,102.371,111.6780,485.666,102.9360,0.001434,...,105.5490,105.5300,105.152,105.4080,105.341,9.87395,176.0200,70.2381,10.7143,NaN


In [8]:
result.rename(columns = {' ': 'Output'}, inplace=True)

In [9]:
result.shape

(3703, 1060)

In [10]:
result_2= result.loc[:,'356.692':'428.919']
result_2.columns
# df_without_calb_None['414.985':'537.294']

Index(['356.692', '357.472', '358.252', '359.031', '359.811', '360.59',
       '361.37', '362.149', '362.928', '363.707', '364.486', '365.266',
       '366.044', '366.823', '367.602', '368.381', '369.16', '369.938',
       '370.717', '371.496', '372.274', '373.052', '373.831', '374.609',
       '375.387', '376.165', '376.943', '377.721', '378.499', '379.277',
       '380.055', '380.833', '381.61', '382.388', '383.165', '383.943',
       '384.72', '385.498', '386.275', '387.052', '387.829', '388.606',
       '389.383', '390.16', '390.937', '391.714', '392.49', '393.267',
       '394.044', '394.82', '395.596', '396.373', '397.149', '397.925',
       '398.702', '399.478', '400.254', '401.03', '401.806', '402.581',
       '403.357', '404.133', '404.908', '405.684', '406.459', '407.235',
       '408.01', '408.786', '409.561', '410.336', '411.111', '411.886',
       '412.661', '413.436', '414.211', '414.985', '415.76', '416.535',
       '417.309', '418.084', '418.858', '419.632', '420.407', 

In [11]:
result_3 =result.loc[:,'Output']
result_2= result.loc[:,'363.707':'423.503']
result_1= result.iloc[:,1:14]


In [12]:
print(result_2.shape, result_3.shape)

(3703, 78) (3703,)


In [13]:
df = pd.concat([result_2,result_3], axis=1)
df.head()


,363.707,364.486,365.266,366.044,366.823,367.602,368.381,369.16,369.938,370.717,...,417.309,418.084,418.858,419.632,420.407,421.181,421.955,422.729,423.503,Output
0,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,W_Calibration
1,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,F_Calibration
2,87.0480,86.5098,86.2694,86.1470,85.8456,85.3556,85.0497,84.6754,84.2657,83.8722,...,78.9688,81.7693,85.1405,87.9776,89.7679,90.9776,91.9985,92.7577,93.4443,W_VK12 | 2_J_None_0.902_SI1
3,62.4444,62.0639,61.7058,61.2996,60.9179,60.5205,60.1136,59.6632,59.1972,58.7010,...,83.3894,85.9996,88.8012,91.5548,93.5495,95.8632,98.2025,100.8680,103.6550,F_VK12 | 2_J_None_0.902_SI1
4,101.9310,101.7240,101.6950,101.6310,101.6350,101.1850,101.1380,100.9720,100.8440,100.6880,...,101.6930,101.8660,102.0850,102.2610,102.3590,102.4060,102.4350,102.4410,102.4610,W_3472A2 | 126_D_None_0.702_VS1


In [14]:
# df.to_excel('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/Processed_data/whole_data.xlsx')

In [92]:
df_without_calb = df[~df['Output'].str.contains(('Calibration|CALIBARTION|CALIBARATION'),case=False, regex=True, na = False)]
df_without_calb.reset_index()
df_without_calb['w/f']= df_without_calb['Output'].str.split('_').str[0]

In [16]:
df_without_calb.head()

,363.707,364.486,365.266,366.044,366.823,367.602,368.381,369.16,369.938,370.717,...,418.084,418.858,419.632,420.407,421.181,421.955,422.729,423.503,Output,w/f
2,87.0480,86.5098,86.2694,86.1470,85.8456,85.3556,85.0497,84.6754,84.2657,83.8722,...,81.7693,85.1405,87.9776,89.7679,90.9776,91.9985,92.7577,93.4443,W_VK12 | 2_J_None_0.902_SI1,W
3,62.4444,62.0639,61.7058,61.2996,60.9179,60.5205,60.1136,59.6632,59.1972,58.7010,...,85.9996,88.8012,91.5548,93.5495,95.8632,98.2025,100.8680,103.6550,F_VK12 | 2_J_None_0.902_SI1,F
4,101.9310,101.7240,101.6950,101.6310,101.6350,101.1850,101.1380,100.9720,100.8440,100.6880,...,101.8660,102.0850,102.2610,102.3590,102.4060,102.4350,102.4410,102.4610,W_3472A2 | 126_D_None_0.702_VS1,W
5,100.9010,100.9810,101.0540,101.0970,101.1460,101.1980,101.2260,101.2470,101.2460,101.2460,...,101.6150,101.7150,101.6780,101.6350,101.2910,101.6420,101.8230,102.0510,F_3472A2 | 126_D_None_0.702_VS1,F
10,84.3379,84.2056,83.7665,83.6529,83.5457,83.4267,83.3603,83.3049,83.4896,83.2446,...,81.5837,83.2302,84.5448,85.3473,85.8796,86.3245,86.6741,86.9831,W_1738A2 | 107_J_LB_None_0.702_SI1,W


In [93]:
df_with_w = df_without_calb[df_without_calb['w/f'] == 'W']


In [94]:
df_with_w['color'] = df_with_w['Output'].str.split('_').str[2]
df_with_w['color'].replace({'E LB':'E', ' K':'K', 'G LB ':'G', 'H ':'H'}, inplace = True)
df_with_w['quality'] = df_with_w['Output'].str.split('_').str[3]


In [95]:
df_with_w['quality'].replace({'lb':'Faint','Benth':'None','brown':'None','Pink':'None', 'LB':'None','LB ':'None', 'Faint YELLOW':'Faint', 'Faint YELLOW':'Faint', 'Faint GREEN':'Faint','  ':'None', 'BROWN':'Faint', 'Lblack':'Faint', 'lb ':'None','Black':'None','faint':'Faint'}, inplace = True)

In [96]:
df_with_w.drop(columns=['Output','w/f'], inplace=True)

In [98]:
df_with_w['quality'].replace({'None':'0', 'Faint':'1', 'Very Strong':'4', 'Medium':'2', 'Strong':'3'}, inplace = True)

In [99]:
df_with_w.head()

,363.707,364.486,365.266,366.044,366.823,367.602,368.381,369.16,369.938,370.717,...,418.084,418.858,419.632,420.407,421.181,421.955,422.729,423.503,color,quality
2,87.0480,86.5098,86.2694,86.1470,85.8456,85.3556,85.0497,84.6754,84.2657,83.8722,...,81.7693,85.1405,87.9776,89.7679,90.9776,91.9985,92.7577,93.4443,J,0
4,101.9310,101.7240,101.6950,101.6310,101.6350,101.1850,101.1380,100.9720,100.8440,100.6880,...,101.8660,102.0850,102.2610,102.3590,102.4060,102.4350,102.4410,102.4610,D,0
10,84.3379,84.2056,83.7665,83.6529,83.5457,83.4267,83.3603,83.3049,83.4896,83.2446,...,81.5837,83.2302,84.5448,85.3473,85.8796,86.3245,86.6741,86.9831,J,0
12,97.5362,97.5079,97.6389,97.3283,97.3299,97.1781,97.1703,97.1557,97.1037,96.7851,...,96.0994,96.9004,97.5015,97.8972,98.1579,98.3211,98.5120,98.6553,G,0
14,101.1990,100.7660,100.6530,100.2860,100.1440,100.1350,100.1250,100.0650,100.1640,99.9450,...,99.9973,100.4380,100.7780,101.0330,101.1920,101.3340,101.4810,101.5450,E,0


In [115]:
df_with_w.iloc[:,-1:]

,quality
2,0
4,0
10,0
12,0
14,0
...,...
3687,0
3691,1
3693,0
3697,0


In [118]:
X = pd.concat([df_with_w.iloc[:,:-2] ,df_with_w.iloc[:,-1:]], axis=1)

In [119]:
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
x_norm = pd.DataFrame(np_scaled, columns = X.columns)

In [120]:
df_with_w.head()

,363.707,364.486,365.266,366.044,366.823,367.602,368.381,369.16,369.938,370.717,...,418.084,418.858,419.632,420.407,421.181,421.955,422.729,423.503,color,quality
2,87.0480,86.5098,86.2694,86.1470,85.8456,85.3556,85.0497,84.6754,84.2657,83.8722,...,81.7693,85.1405,87.9776,89.7679,90.9776,91.9985,92.7577,93.4443,J,0
4,101.9310,101.7240,101.6950,101.6310,101.6350,101.1850,101.1380,100.9720,100.8440,100.6880,...,101.8660,102.0850,102.2610,102.3590,102.4060,102.4350,102.4410,102.4610,D,0
10,84.3379,84.2056,83.7665,83.6529,83.5457,83.4267,83.3603,83.3049,83.4896,83.2446,...,81.5837,83.2302,84.5448,85.3473,85.8796,86.3245,86.6741,86.9831,J,0
12,97.5362,97.5079,97.6389,97.3283,97.3299,97.1781,97.1703,97.1557,97.1037,96.7851,...,96.0994,96.9004,97.5015,97.8972,98.1579,98.3211,98.5120,98.6553,G,0
14,101.1990,100.7660,100.6530,100.2860,100.1440,100.1350,100.1250,100.0650,100.1640,99.9450,...,99.9973,100.4380,100.7780,101.0330,101.1920,101.3340,101.4810,101.5450,E,0


In [122]:
x = x_norm
Y = df_with_w.iloc[:,-2:-1]

In [89]:
Y.value_counts()

0    676
1    324
4    165
2    100
3      3
Name: quality, dtype: int64

In [ ]:
balanced = class_weight.compute_class_weight(class_weight='balanced', classes=df_without_calb_None['color'].value_counts().keys(),y=Y)
balanced = dict(zip(df_without_calb_None['color'].value_counts().keys(),balanced))
balanced

In [124]:
X_train, X_test, Y_train, Y_test = train_test_split(x,Y, test_size = 0.2, random_state =42, stratify = Y)

In [125]:
stratifiedkf=StratifiedKFold(n_splits=5)

In [126]:
light_gb_classifier = lgb.LGBMClassifier()
grid_values = {'n_estimators':[*range(50, 500, 20)], 'max_depth':[*range(1,10,1)], 'max_features':[*range(1,100,1)],'min_samples_split':np.linspace(0.1, 1.0, 10, endpoint=True),'min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True)}
boost_classifier = RandomizedSearchCV(light_gb_classifier, param_distributions= grid_values , cv = stratifiedkf)

In [127]:
decision_classifier = DecisionTreeClassifier()
grid_values = {'max_depth':[*range(1,10,1)], 'max_features':[*range(1,100,1)],'min_samples_split':np.linspace(0.1, 1.0, 10, endpoint=True),'min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True)}
decision_tree_classifier = RandomizedSearchCV(decision_classifier, param_distributions= grid_values , cv = 5)

In [128]:
forest_classifier = RandomForestClassifier(class_weight = balanced)
grid_values = {'n_estimators':[*range(50, 500, 20)], 'max_depth':[*range(1,10,1)], 'max_features':[*range(1,100,1)],'min_samples_split':np.linspace(0.1, 1.0, 10, endpoint=True),'min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True)}
random_classifier = RandomizedSearchCV(forest_classifier, param_distributions= grid_values , cv = stratifiedkf)

In [139]:
boost_classfier_ = GradientBoostingClassifier()
grid_values = {'n_estimators':[*range(50, 500, 20)], 'max_depth':[*range(1,10,1)], 'max_features':[*range(1,100,1)],'min_samples_split':np.linspace(0.1, 1.0, 10, endpoint=True),'min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True)}
boost_classifier = RandomizedSearchCV(boost_classfier_, param_distributions= grid_values , cv = stratifiedkf)
# score=cross_val_scoreida(logreg,X,Y,cv=stratifiedkf)
# print("Cross Validation Scores are {}".format(score))
# print("Average Cross Valtion score :{}".format(score.mean()))

In [130]:
boost_classfier_ = xgb.XGBClassifier(enabale_categorical = True)
grid_values = {'n_estimators':[*range(50, 500, 20)], 'max_depth':[*range(1,10,1)], 'max_features':[*range(1,100,1)],'min_samples_split':np.linspace(0.1, 1.0, 10, endpoint=True),'min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True)}
xgboost_classifier = RandomizedSearchCV(boost_classfier_, param_distributions= grid_values , cv = stratifiedkf)
# score=cross_val_scoreida(logreg,X,Y,cv=stratifiedkf)
# print("Cross Validation Scores are {}".format(score))
# print("Average Cross Valtion score :{}".format(score.mean()))

In [131]:
pickle.dump(random_classifier, open('/home/pankaj_v/Documents/Data_analysis_dixit/Models/random_forest_model.pkl', 'wb'))

In [140]:
boost_classifier.fit(X_train, Y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   estimator=GradientBoostingClassifier(),
                   param_distributions={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8,
                                                      9],
                                        'max_features': [1, 2, 3, 4, 5, 6, 7, 8,
                                                         9, 10, 11, 12, 13, 14,
                                                         15, 16, 17, 18, 19, 20,
                                                         21, 22, 23, 24, 25, 26,
                                                         27, 28, 29, 30, ...],
                                        'min_samples_leaf': array([0.1, 0.2, 0.3, 0.4, 0.5]),
                                        'min_samples_split': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'n_estimators': [50, 70, 90, 110, 130,
                                                         150, 170, 190, 210,
                                                         230, 250, 270, 290,
                                                         310, 330, 350, 370,
                                                         390, 410, 430, 450,
                                                         470, 490]})

In [141]:
random_classifier.fit(X_train, Y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   estimator=RandomForestClassifier(class_weight={'D': 0.7273706896551724,
                                                                  'E': 0.9698275862068966,
                                                                  'F': 0.7740825688073395,
                                                                  'G': 0.6643700787401575,
                                                                  'H': 1.0817307692307692,
                                                                  'I': 1.016566265060241,
                                                                  'J': 2.057926829268293,
                                                                  'K': 2.4816176470588234}),
                   param_distributions={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8,
                                                      9],
                                        'max_features': [1, 2, 3, 4, 5, 6, 7, 8,
                                                         9, 10, 11, 12, 13, 14,
                                                         15, 16, 17, 18, 19, 20,
                                                         21, 22, 23, 24, 25, 26,
                                                         27, 28, 29, 30, ...],
                                        'min_samples_leaf': array([0.1, 0.2, 0.3, 0.4, 0.5]),
                                        'min_samples_split': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'n_estimators': [50, 70, 90, 110, 130,
                                                         150, 170, 190, 210,
                                                         230, 250, 270, 290,
                                                         310, 330, 350, 370,
                                                         390, 410, 430, 450,
                                                         470, 490]})

In [142]:
light_gb_classifier.fit(X_train, Y_train)

LGBMClassifier()

In [ ]:
pickle_model = pickle.load(open('/home/pankaj_v/Documents/Data_analysis_dixit/Models/lgb_model.pkl','rb'))

In [ ]:
boost_classifier.best_params_


{'n_estimators': 450,
 'min_samples_split': 0.30000000000000004,
 'min_samples_leaf': 0.1,
 'max_features': 69,
 'max_depth': 7}

In [149]:
boost_new = boost_classifier.best_estimator_
random_new = random_classifier.best_estimator_
light_new = light_gb_classifier


In [ ]:
clf_new.feature_importances_

NameError: name 'clf_new' is not defined

In [ ]:
sfs1 = sfs(light_new,
           k_features=5,
           forward=True,
           floating=False,
           verbose=2,
           scoring='accuracy',
           cv=5)

In [160]:
train_preds_boost = boost_new.predict(X_train)
test_preds_boost = boost_new.predict(X_test)

In [154]:
train_preds_random = random_new.predict(X_train)
test_preds_random = random_new.predict(X_test)

In [155]:
train_preds_light = light_new.predict(X_train)
test_preds_light = light_new.predict(X_test)

In [156]:
# pd.DataFrame(clf.cv_results_).iloc[:,9:]

AttributeError: 'LGBMClassifier' object has no attribute 'cv_results_'

In [161]:
print(classification_report(Y_test, test_preds_boost))

              precision    recall  f1-score   support

           D       0.74      0.59      0.66        39
           E       0.44      0.50      0.47        32
           F       0.49      0.56      0.52        36
           G       0.43      0.44      0.43        41
           H       0.32      0.31      0.32        29
           I       0.35      0.38      0.36        32
           J       0.33      0.25      0.29        24
           K       0.54      0.65      0.59        20
           M       0.00      0.00      0.00         1

    accuracy                           0.46       254
   macro avg       0.41      0.41      0.40       254
weighted avg       0.46      0.46      0.46       254



In [158]:
print(classification_report(Y_test, test_preds_random))


              precision    recall  f1-score   support

           D       0.45      0.64      0.53        39
           E       0.22      0.22      0.22        32
           F       0.00      0.00      0.00        36
           G       0.32      0.49      0.39        41
           H       1.00      0.07      0.13        29
           I       0.00      0.00      0.00        32
           J       0.11      0.08      0.10        24
           K       0.23      0.95      0.37        20
           M       0.00      0.00      0.00         1

    accuracy                           0.30       254
   macro avg       0.26      0.27      0.19       254
weighted avg       0.29      0.30      0.22       254



In [159]:
print(classification_report(Y_test, test_preds_light))

              precision    recall  f1-score   support

           D       0.69      0.62      0.65        39
           E       0.37      0.34      0.35        32
           F       0.50      0.56      0.53        36
           G       0.45      0.44      0.44        41
           H       0.22      0.24      0.23        29
           I       0.43      0.47      0.45        32
           J       0.42      0.46      0.44        24
           K       0.75      0.60      0.67        20
           M       0.00      0.00      0.00         1

    accuracy                           0.46       254
   macro avg       0.42      0.41      0.42       254
weighted avg       0.47      0.46      0.47       254



In [ ]:
print(round(accuracy_score(Y_test, test_preds)*100,2))

86.76


In [ ]:
confusion_matrix(Y_test, test_preds)


array([[21,  1,  1,  0,  0,  0,  0,  0],
       [ 6,  7,  3,  1,  0,  0,  0,  0],
       [ 2,  4, 13,  2,  0,  1,  0,  0],
       [ 3,  1,  2, 11,  3,  4,  1,  0],
       [ 0,  0,  0,  8,  6,  2,  0,  0],
       [ 0,  0,  1,  1,  4,  9,  2,  0],
       [ 0,  0,  0,  0,  1,  0,  3,  4],
       [ 0,  0,  0,  0,  0,  1,  1,  5]])